In [9]:
import pandas as pd
import numpy as np
from datetime import date, datetime
import locale



In [10]:
df1 = pd.read_csv("../input/atus_anual_2020.csv", index_col = False)
df2 = pd.read_csv("../input/atus_anual_2021.csv", index_col = False)
df3 = pd.read_csv("../input/atus_anual_2022.csv", index_col = False)

1. Cargue en un Dataframe los datos de los accidentes de los últimos 3 años

In [11]:
df_unido = [df1, df2, df3]
df_unido = pd.concat(df_unido)

2. Agregue una columna tipo date que englobe las columnas correspondientes a Año, Mes, Dia, Hora y Minuto.

In [12]:
df_unido['FECHA'] = df_unido.apply(lambda row: f"{row['ANIO']}-{row['MES']}-{row['ID_DIA']} {row['ID_HORA']}:{row['ID_MINUTO']}", axis=1)

df_unido['FECHA'] = pd.to_datetime(df_unido['FECHA'], format='%Y-%m-%d %H:%M')

df_unido = df_unido.drop(['ANIO', 'MES', 'ID_DIA', 'ID_HORA', 'ID_MINUTO','DIASEMANA'], axis=1)
df_unido.head()

,COBERTURA,ID_ENTIDAD,ID_MUNICIPIO,URBANA,SUBURBANA,TIPACCID,AUTOMOVIL,CAMPASAJ,MICROBUS,PASCAMION,...,PEATHERIDO,CICLMUERTO,CICLHERIDO,OTROMUERTO,OTROHERIDO,NEMUERTO,NEHERIDO,CLASACC,ESTATUS,FECHA
0,Municipal,1,1,Accidente en intersección,Sin accidente en esta zona,Colisión con vehículo automotor,1,1,0,0,...,0,0,0,0,0,0,0,Sólo daños,Cifras Definitivas,2020-01-01 00:00:00
1,Municipal,1,1,Accidente en intersección,Sin accidente en esta zona,Colisión con objeto fijo,1,0,0,0,...,0,0,0,0,0,0,0,Sólo daños,Cifras Definitivas,2020-01-01 00:00:00
2,Municipal,1,1,Accidente en intersección,Sin accidente en esta zona,Colisión con motocicleta,1,0,0,0,...,0,0,0,0,0,0,0,Sólo daños,Cifras Definitivas,2020-01-01 00:10:00
3,Municipal,1,1,Accidente en intersección,Sin accidente en esta zona,Colisión con vehículo automotor,2,0,0,0,...,0,0,0,0,0,0,0,Sólo daños,Cifras Definitivas,2020-01-01 02:00:00
4,Municipal,1,1,Accidente en intersección,Sin accidente en esta zona,Colisión con vehículo automotor,2,0,0,0,...,0,0,0,0,0,0,0,Sólo daños,Cifras Definitivas,2020-01-01 02:10:00


3. Agregue una columna para Área cuyos valores serán Urbana o Suburbana dependiendo del área donde ocurrió el accidente.
4. Agregue una columna Zona cuyos valores serán la zona donde ocurrió el accidente
5. Elimine las columnas URBANA y SUBURBANA

In [13]:
df_unido['ZONA'] = df_unido.apply(lambda row: row['URBANA'] if row['URBANA'] != 'Sin accidente en esta zona' else (row['SUBURBANA'] if row['SUBURBANA'] != 'Sin accidente en esta zona' else None), axis=1)
df_unido['URBANA'] = (df_unido['URBANA']=='Sin accidente en esta zona').astype(int)
print(df_unido.groupby("URBANA"))
df_unido['URBANA'] = df_unido['URBANA'].apply(lambda x: 'Suburbana' if x == 1 else 'Urbana')
df_unido = df_unido.drop(['SUBURBANA'], axis=1)
df_unido = df_unido.rename(columns = {'URBANA' : 'AREA'})
df_unido.head()

,COBERTURA,ID_ENTIDAD,ID_MUNICIPIO,AREA,TIPACCID,AUTOMOVIL,CAMPASAJ,MICROBUS,PASCAMION,OMNIBUS,...,CICLMUERTO,CICLHERIDO,OTROMUERTO,OTROHERIDO,NEMUERTO,NEHERIDO,CLASACC,ESTATUS,FECHA,ZONA
0,Municipal,1,1,Urbana,Colisión con vehículo automotor,1,1,0,0,0,...,0,0,0,0,0,0,Sólo daños,Cifras Definitivas,2020-01-01 00:00:00,Accidente en intersección
1,Municipal,1,1,Urbana,Colisión con objeto fijo,1,0,0,0,0,...,0,0,0,0,0,0,Sólo daños,Cifras Definitivas,2020-01-01 00:00:00,Accidente en intersección
2,Municipal,1,1,Urbana,Colisión con motocicleta,1,0,0,0,0,...,0,0,0,0,0,0,Sólo daños,Cifras Definitivas,2020-01-01 00:10:00,Accidente en intersección
3,Municipal,1,1,Urbana,Colisión con vehículo automotor,2,0,0,0,0,...,0,0,0,0,0,0,Sólo daños,Cifras Definitivas,2020-01-01 02:00:00,Accidente en intersección
4,Municipal,1,1,Urbana,Colisión con vehículo automotor,2,0,0,0,0,...,0,0,0,0,0,0,Sólo daños,Cifras Definitivas,2020-01-01 02:10:00,Accidente en intersección


7. Resuma los accidentes del último año por tipo y que porcentaje representan

In [14]:
ultimo_año = df_unido['FECHA'].dt.year.max()

datos_ultimo_año = df_unido[df_unido['FECHA'].dt.year == ultimo_año]

tipo_accidente_counts = datos_ultimo_año['TIPACCID'].value_counts().reset_index()
tipo_accidente_counts.columns = ['Tipo de Accidente', 'Count']

total_accidentes_ultimo_año = tipo_accidente_counts['Count'].sum()
tipo_accidente_counts['Porcentaje sobre el Total'] = (tipo_accidente_counts['Count'] / total_accidentes_ultimo_año) * 100

print(tipo_accidente_counts)

                        Tipo de Accidente   Count  Porcentaje sobre el Total
0         Colisión con vehículo automotor  229940                  58.552112
1                Colisión con motocicleta   53629                  13.656133
2                Colisión con objeto fijo   47033                  11.976522
3                        Certificado cero   15479                   3.941585
4   Colisión con peatón (atropellamiento)   11066                   2.817855
5                               Volcadura   10911                   2.778386
6                       Salida del camino   10857                   2.764635
7                                    Otro    6325                   1.610603
8                   Colisión con ciclista    4048                   1.030786
9                       Caída de pasajero    1784                   0.454279
10                    Colisión con animal    1022                   0.260243
11                               Incendio     348                   0.088615

8. Obtenga la cantidad de muertos y lesionados de los últimos 3 años

In [15]:
Muertos_y_lesion = pd.DataFrame()

total_m_y_l = df_unido[['CONDMUERTO','PASAMUERTO','PEATMUERTO', 'CICLMUERTO', 'OTROMUERTO', 'NEMUERTO', 'CONDHERIDO','PASAHERIDO', 'PEATHERIDO', 'CICLHERIDO', 'OTROHERIDO', 'NEHERIDO']].sum()

Muertos_y_lesion['total'] = total_m_y_l

Muertos_y_lesion['categorias'] = df_unido[['CONDMUERTO','PASAMUERTO','PEATMUERTO', 'CICLMUERTO', 'OTROMUERTO', 'NEMUERTO', 'CONDHERIDO','PASAHERIDO', 'PEATHERIDO', 'CICLHERIDO', 'OTROHERIDO', 'NEHERIDO']].columns

Muertos_y_lesion['porcentaje sobre total'] = (Muertos_y_lesion['total'] / Muertos_y_lesion['total'].sum()) * 100


print(Muertos_y_lesion)

             total  categorias  porcentaje sobre total
CONDMUERTO    7050  CONDMUERTO                2.718754
PASAMUERTO    3088  PASAMUERTO                1.190853
PEATMUERTO    2689  PEATMUERTO                1.036983
CICLMUERTO     433  CICLMUERTO                0.166982
OTROMUERTO     148  OTROMUERTO                0.057075
NEMUERTO         0    NEMUERTO                0.000000
CONDHERIDO  127628  CONDHERIDO               49.218310
PASAHERIDO   78588  PASAHERIDO               30.306583
PEATHERIDO   31949  PEATHERIDO               12.320774
CICLHERIDO    6787  CICLHERIDO                2.617331
OTROHERIDO     950  OTROHERIDO                0.366357
NEHERIDO         0    NEHERIDO                0.000000


9. ¿Cómo se compara el número de accidentes de cada año contra el anterior? ¿En qué porcentaje varió?

In [16]:
df_unido['FECHA'] = pd.to_datetime(df_unido['FECHA'])

df_unido['AÑO'] = df_unido['FECHA'].dt.year

total_accidentes_por_año = df_unido['AÑO'].value_counts().reset_index()
total_accidentes_por_año.columns = ['AÑO', 'Total_Accidentes']

total_accidentes_por_año['Porcentaje'] = (total_accidentes_por_año['Total_Accidentes'] / total_accidentes_por_año['Total_Accidentes'].sum()) * 100

datos = df_unido.drop('AÑO', axis = 1)
print(total_accidentes_por_año)

    AÑO  Total_Accidentes  Porcentaje
0  2022            392710   36.802612
1  2021            356315   33.391874
2  2020            318046   29.805514
